# HateXplain

https://huggingface.co/Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two


The model is used for classifying a text as Abusive (Hatespeech and Offensive) or Normal. The model is trained using data from Gab and Twitter and Human Rationales were included as part of the training data to boost the performance. The model also has a rationale predictor head that can predict the rationales given an abusive sentence.

The dataset and models are available here: https://github.com/punyajoy/HateXplain

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BatchEncoding
### from models.py
from models import *
import os
import shutil

import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from tensorflow.python.ops.numpy_ops import np_config

tf.get_logger().setLevel('ERROR')

seed = 42
batch_size = 10
test_size = 30
MAX_LENGTH = 128

In [2]:
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two")
model = Model_Rational_Label.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two")

In [3]:
# loading in testing data set
customer_data = pd.read_csv("datasets\\Customer support tweets\\twcs.csv").drop(columns=["tweet_id", "author_id", "inbound", "created_at", "response_tweet_id","in_response_to_tweet_id"])

In [4]:
# Source https://towardsdatascience.com/hugging-face-transformers-fine-tuning-distilbert-for-binary-classification-tasks-490f1d192379

def batch_encode(tokenizer, texts, batch_size=256, max_length=MAX_LENGTH):
    """""""""
    A function that encodes a batch of texts and returns the texts'
    corresponding encodings and attention masks that are ready to be fed 
    into a pre-trained transformer model.
    
    Input:
        - tokenizer:   Tokenizer object from the PreTrainedTokenizer Class
        - texts:       List of strings where each string represents a text
        - batch_size:  Integer controlling number of texts in a batch
        - max_length:  Integer controlling max number of words to tokenize in a given text
    Output:
        - input_ids:       sequence of texts encoded as a tf.Tensor object
        - attention_mask:  the texts' attention mask encoded as a tf.Tensor object
    """""""""
    
    input_ids = []
    attention_mask = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             padding='longest', #implements dynamic padding
                                             return_attention_mask=True,
                                             return_token_type_ids=False
                                             )
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])
    
    
    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)
    

In [31]:
data = customer_data.text.sample(test_size).values.tolist()
input_ids = []
attention_mask = []


batchencoder = (tokenizer(data[0:10], return_tensors="pt", padding=True))

print(batchencoder['input_ids'])
#                        padding='longest', #implements dynamic padding
#                        return_attention_mask=True,
#                        return_token_type_ids=False).convert_to_tensors("tf")

# input_ids = batchencoder.convert_to_tensors("tf")
# max_length = np.max(list(map(len, data)))
# print(max_length)
# input_ids, attention_mask = batch_encode(tokenizer=tokenizer, texts=data, max_length=max_length)
# print(input_ids)

tensor([[  101,  1030, 23090,  2581, 21472, 22091,  2860,  4283,  1010,  4074,
          1012,  1045,  2572,  2383,  1037,  2204, 15060,  1012,  1011, 10225,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  1030, 16327, 21619,  2549,  2057,  1005,  2128,  3374,  2000,
          2963,  2023,  1010,  4575,  1012,  2057,  2079,  3246,  2115,  3462,
          2067,  2003,  2172,  2062, 22249,  2174,  1012,  4283,  2005,  1996,
         12247,   999,  1034,  8840,  9215,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  1030, 24559, 22407,  2629,  4931,   999,  2057,  1521,  2128,
          3374,  2000,  2963,  2008,  1012,  4638,  2041, 16770,  1024,  1013,
          1013,  1056,  1012,  2522,  1013,  1053,  7295,  2509,

In [32]:
prediction_logits, _ = model(input_ids=batchencoder['input_ids'],attention_mask=batchencoder['attention_mask'])

In [34]:
torch.argmax(prediction_logits, dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])